In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/sensei-fs/users/nikhilanand/8steering_llama/sllama_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.07it/s]


In [2]:
tokenizer.convert_tokens_to_ids("<pad>")

0

In [4]:
tokenizer.convert_ids_to_tokens(0)

'<pad>'

In [2]:
from llama_wrapper import LlamaWrapper
import os
HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")

In [3]:
model_wrapper = LlamaWrapper(
    HUGGINGFACE_TOKEN,
    size="7b",
    use_chat=True,
    override_model_weights_path=None,
    use_latest = False,
    use_mistral = True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.03it/s]


In [4]:
model_wrapper.model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x BlockOutputWrapper(
        (block): Gemma2DecoderLayer(
          (self_attn): AttnWrapper(
            (attn): Gemma2SdpaAttention(
              (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
              (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
              (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
              (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
              (rotary_emb): Gemma2RotaryEmbedding()
            )
          )
          (mlp): Gemma2MLP(
            (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
            (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
            (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
            (act_fn): PytorchGELUTanh()
          )


In [5]:
model_wrapper.model_name_path

'google/gemma-2-2b-it'

In [6]:
from prompting_with_steering import process_item_ab

In [7]:
output = {
    "question": "Context: <P> The series stars Bob Neuwirth as Shaun Murphy , a young surgical resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes . </P>\nQuestion: who is the actor that plays dr. sean murphy?\n\nChoices:\n (A) Bob Neuwirth\n (B) Freddie Highmore",
    "answer_matching_behavior": "(A)",
    "answer_not_matching_behavior": "(B)",
    "a_prob": 0.2031976729631424,
    "b_prob": 0.30618181824684143
}

item = {
    "question": "Context: <P> The series stars Bob Neuwirth as Shaun Murphy , a young surgical resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes . </P>\nQuestion: who is the actor that plays dr. sean murphy?\n\nChoices:\n (A) Bob Neuwirth\n (B) Freddie Highmore",
    "answer_matching_behavior": "(A)",
    "answer_not_matching_behavior": "(B)"
}

In [8]:
a_token_id = model_wrapper.tokenizer.convert_tokens_to_ids("A")
b_token_id = model_wrapper.tokenizer.convert_tokens_to_ids("B")

In [9]:
process_item_ab(
    item,
    model_wrapper,
    None,
    a_token_id,
    b_token_id
)

google/gemma-2-2b-it


{'question': 'Context: <P> The series stars Bob Neuwirth as Shaun Murphy , a young surgical resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes . </P>\nQuestion: who is the actor that plays dr. sean murphy?\n\nChoices:\n (A) Bob Neuwirth\n (B) Freddie Highmore',
 'answer_matching_behavior': '(A)',
 'answer_not_matching_behavior': '(B)',
 'a_prob': 0.9958133101463318,
 'b_prob': 0.0005368636921048164}

In [107]:
model_wrapper.tokenizer.convert_ids_to_tokens(a_token_id)

'A'

In [108]:
import torch
import torch.nn.functional as F
from utils.tokenize import template_gemma

In [109]:
inp_text = template_gemma(item['question'],"(")
print("Inp text",inp_text)
print("*****")
input_tokens = torch.tensor([model_wrapper.tokenizer(inp_text).input_ids]).to("cuda")
with torch.no_grad():
    logits = model_wrapper.model(input_tokens).logits[0,-1,:]
    probs = F.softmax(logits)
    max_token_id = torch.topk(probs,k=10)
li = list(max_token_id.indices)
li2 = [item.item() for item in li]
print("Top 10 Tokens:",model_wrapper.tokenizer.convert_ids_to_tokens(li2))
print("*****")
print("Probs:",[probs[item].item() for item in li2])
print("*****")

Inp text <start_of_turn>user
Context: <P> The series stars Bob Neuwirth as Shaun Murphy , a young surgical resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes . </P>
Question: who is the actor that plays dr. sean murphy?

Choices:
 (A) Bob Neuwirth
 (B) Freddie Highmore<end_of_turn>
<start_of_turn>assistant
 (
*****
Top 10 Tokens: ['A', '**', '▁**', 'B', '▁A', 'a', 'Answer', '**(', 'The', 'C']
*****
Probs: [0.9958133101463318, 0.0013438839232549071, 0.0006383450818248093, 0.0005368636921048164, 0.0003909683146048337, 0.0002518380351830274, 0.0002377637865

/tmp/ipykernel_1080720/3323877758.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)


In [110]:
question: str = item["question"]
model_output = model_wrapper.get_logits_from_text(
    user_input=question, model_output="(", system_prompt=None
)
probs = F.softmax(model_output[0,-1,:])
max_token_id = torch.topk(probs,k=10)
li = list(max_token_id.indices)
li2 = [item.item() for item in li]
print("Top 10 Tokens:",model_wrapper.tokenizer.convert_ids_to_tokens(li2))
print("*****")
print("Probs:",[probs[item].item() for item in li2])
print("*****")

google/gemma-2-2b-it
Top 10 Tokens: ['B', 'A', 'Answer', 'Correct', 'C', 'The', 'correct', '\n', 'Please', 'I']
*****
Probs: [0.30618181824684143, 0.2031976729631424, 0.1453975886106491, 0.08620653301477432, 0.06549584865570068, 0.015432855114340782, 0.011209656484425068, 0.0085744708776474, 0.008086396381258965, 0.007474529091268778]
*****


/tmp/ipykernel_1080720/1583116637.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(model_output[0,-1,:])


In [111]:
from utils.tokenize import tokenize_llama_chat

In [112]:
def get_logits_from_text(user_input: str, model_output = None, system_prompt = None):
    tokens = tokenize_llama_chat(
        tokenizer=model_wrapper.tokenizer, user_input=user_input, model_output=model_output, system_prompt=system_prompt
    )
    print(tokens)
    tokens = torch.tensor(tokens).unsqueeze(0).to(model_wrapper.device)
    logits = model_wrapper.model(tokens).logits
    return logits

In [113]:
model_wrapper.tokenizer.name_or_path

'google/gemma-2-2b-it'

In [114]:
def tokenize_llama_chat2(
    tokenizer,
    user_input: str,
    model_output: str = None,
    system_prompt: str = None
):
    print(tokenizer.name_or_path)
    if tokenizer.name_or_path=="google/gemma-2-2b-it":
        input_content = template_gemma(user_input, model_output, system_prompt)
        return tokenizer.encode(input_content)

In [115]:
question: str = item["question"]
tokens = tokenize_llama_chat2(
    tokenizer=model_wrapper.tokenizer, user_input=question, model_output="(", system_prompt=None
)
print(tokens)
tokens = torch.tensor(tokens).unsqueeze(0).to(model_wrapper.device)
model_output = model_wrapper.model(tokens).logits
probs = F.softmax(model_output[0,-1,:])
max_token_id = torch.topk(probs,k=10)
li = list(max_token_id.indices)
li2 = [item.item() for item in li]
print("Top 10 Tokens:",model_wrapper.tokenizer.convert_ids_to_tokens(li2))
print("*****")
print("Probs:",[probs[item].item() for item in li2])
print("*****")

google/gemma-2-2b-it
[2, 106, 1645, 108, 2930, 235292, 968, 235295, 235313, 714, 4100, 8995, 11270, 17328, 23361, 489, 685, 106515, 30309, 1688, 476, 3486, 32635, 19099, 675, 65811, 578, 211262, 25971, 696, 3250, 8289, 997, 235265, 9191, 107069, 11151, 954, 142636, 7536, 1688, 34218, 64514, 1688, 1074, 19913, 5957, 235261, 1688, 39271, 67000, 1688, 9300, 35753, 1688, 9135, 67036, 1688, 578, 13346, 13845, 6738, 2170, 1170, 2343, 575, 573, 1500, 954, 714, 4100, 4956, 476, 2507, 18576, 16784, 696, 23974, 1452, 476, 4509, 22215, 4100, 1498, 780, 3124, 4830, 696, 44132, 42384, 36278, 575, 235248, 235284, 235276, 235274, 235308, 2161, 714, 5655, 18570, 729, 11538, 577, 4100, 575, 2782, 235248, 235284, 235276, 235274, 235324, 954, 1993, 5424, 235248, 235304, 1688, 235248, 235284, 235276, 235274, 235324, 1688, 23974, 15532, 908, 573, 4100, 604, 476, 2247, 3891, 576, 235248, 235274, 235321, 24958, 954, 2692, 235295, 235313, 108, 9413, 235292, 1064, 603, 573, 14250, 674, 12258, 4136, 235265, 365

/tmp/ipykernel_1080720/158134904.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(model_output[0,-1,:])


In [116]:
inp_text = template_gemma(item['question'],"(")
print("Inp text",inp_text)
print("*****")
input_tokens = torch.tensor([model_wrapper.tokenizer(inp_text).input_ids]).to("cuda")
with torch.no_grad():
    logits = model_wrapper.model(input_tokens).logits[0,-1,:]
    probs = F.softmax(logits)
    max_token_id = torch.topk(probs,k=10)
li = list(max_token_id.indices)
li2 = [item.item() for item in li]
print("Top 10 Tokens:",model_wrapper.tokenizer.convert_ids_to_tokens(li2))
print("*****")
print("Probs:",[probs[item].item() for item in li2])
print("*****")

Inp text <start_of_turn>user
Context: <P> The series stars Bob Neuwirth as Shaun Murphy , a young surgical resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes . </P>
Question: who is the actor that plays dr. sean murphy?

Choices:
 (A) Bob Neuwirth
 (B) Freddie Highmore<end_of_turn>
<start_of_turn>assistant
 (
*****
Top 10 Tokens: ['A', '**', '▁**', 'B', '▁A', 'a', 'Answer', '**(', 'The', 'C']
*****
Probs: [0.9958133101463318, 0.0013438839232549071, 0.0006383450818248093, 0.0005368636921048164, 0.0003909683146048337, 0.0002518380351830274, 0.0002377637865

/tmp/ipykernel_1080720/3323877758.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)
